# Topic Modeling with gensim

In [1]:
from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import pickle

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [3]:
# gensim
import gensim
from gensim import corpora, models, similarities, matutils
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from gensim import corpora, models, similarities, matutils
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.models import Phrases
from gensim.models.phrases import Phraser

# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import pickle
import numpy as np
import pandas as pd

C:\Users\moham\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\moham\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


## Document Preprocessing
We'll need to generate a term-document matrix of word (token) counts for use in LDA.

We'll use `sklearn`'s `CountVectorizer` to generate our term-document matrix of counts. We'll make use of a few parameters to accomplish the following preprocessing of the text documents all within the `CountVectorizer`:
* `analyzer=word`: Tokenize by word
* `ngram_range=(1,2)`: Keep all 1 and 2-word grams
* `stop_words=english`: Remove all English stop words
* `token_pattern=\\b[a-z][a-z]+\\b`: Match all tokens with 2 or more (strictly) alphabet characters

In [22]:
#Open Corpus of Bitcoin News Articles
with open('../news_articles_processing/data_frames/bitcoin_news_df_processed_for_modeling.pickle', 'rb') as file:
     bitcoin_news_df = pickle.load(file)

### Prepare Bigram Text

In [7]:
bigram_text = Phrases(bitcoin_news_df.tokenized_text, min_count=2, threshold = 80);
bigram_mod_text = Phraser(bigram_text);

2019-06-12 01:28:48,031 : INFO : collecting all words and their counts
2019-06-12 01:28:48,032 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-06-12 01:28:51,990 : INFO : PROGRESS: at sentence #10000, processed 2608196 words and 1414836 word types
2019-06-12 01:28:54,755 : INFO : collected 2238480 word types from a corpus of 4473232 words (unigram + bigrams) and 17048 sentences
2019-06-12 01:28:54,755 : INFO : using 2238480 counts as vocab in Phrases<0 vocab, min_count=2, threshold=80, max_vocab_size=40000000>
2019-06-12 01:28:54,756 : INFO : source_vocab length 2238480
2019-06-12 01:29:13,208 : INFO : Phraser built with 25218 25218 phrasegrams


In [23]:
words_bigrams_news_text = pd.Series([bigram_mod_text[doc] for doc in bitcoin_news_df.tokenized_text])

In [25]:
news_extracted = words_bigrams_news_text.apply(lambda x: ' '.join(x))

## Convert to document-term matrix
Next, the raw documents are converted into document-term matrix, possibly as raw counts or in TF-IDF form.

In [28]:
#Bigrams
tf_vectorizer_bi = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)

dtm_tf_bi = tf_vectorizer_bi.fit_transform(news_extracted)
print(dtm_tf_bi.shape)

tfidf_vectorizer_bi = TfidfVectorizer(**tf_vectorizer_bi.get_params())
dtm_tfidf_bi = tfidf_vectorizer_bi.fit_transform(news_extracted)
print(dtm_tfidf_bi.shape)

(17048, 14468)


C:\Users\moham\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1616: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  UserWarning)


(17048, 14468)


## Fit Latent Dirichlet Allocation models
Finally, the LDA models are fitted.

In [29]:
#Bigrams
# for TF DTM
lda_tf_bi = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tf_bi.fit(dtm_tf_bi)

# for TFIDF DTM
lda_tfidf_bi = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf_bi.fit(dtm_tfidf_bi)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

## Visualizing the models with pyLDAvis

In [30]:
bi_real = pyLDAvis.sklearn.prepare(lda_tf_bi, dtm_tf_bi, tf_vectorizer_bi)
bi_real

C:\Users\moham\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18    -0.072019 -0.034639       1        1  13.150162
5     -0.089811  0.114955       2        1   8.785708
19     0.041230 -0.030993       3        1   8.221394
10    -0.107117  0.066888       4        1   7.577843
15    -0.090758 -0.068406       5        1   7.267126
11     0.128350  0.205978       6        1   6.953296
14    -0.127029  0.069985       7        1   6.424143
0     -0.100321  0.070654       8        1   4.926423
2     -0.067865 -0.024045       9        1   4.807413
3     -0.162101 -0.027955      10        1   4.616362
17     0.167806  0.026816      11        1   4.175490
12    -0.113565 -0.034767      12        1   3.577150
1      0.131193 -0.054692      13        1   3.381218
4      0.155958 -0.134313      14        1   3.308211
6     -0.096582 -0.018346      15        1   3.119291
13    -0.020042 -0.032455      16        1   2.605805
9      0.163473  0.271071      17        1   1.927425
16     0.079171 -0.070686      18        1   1.888725
8      0.123825 -0.169188      19        1   1.877083
7      0.056203 -0.125861      20        1   1.409731, topic_info=      Category          Freq              Term         Total  loglift  logprob
9902   Default  14940.000000             price  14940.000000  30.0000  30.0000
1454   Default  30319.000000        blockchain  30319.000000  29.0000  29.0000
4800   Default  12307.000000          exchange  12307.000000  28.0000  28.0000
3221   Default  16677.000000    cryptocurrency  16677.000000  27.0000  27.0000
8237   Default   6050.000000            mining   6050.000000  26.0000  26.0000
12841  Default  17797.000000        technology  17797.000000  25.0000  25.0000
2573   Default  10356.000000           company  10356.000000  24.0000  24.0000
13772  Default  10935.000000             users  10935.000000  23.0000  23.0000
4695   Default   4244.000000          ethereum   4244.000000  22.0000  22.0000
4805   Default   6535.000000         exchanges   6535.000000  21.0000  21.0000
13198  Default   8957.000000           trading   8957.000000  20.0000  20.0000
11241  Default  14315.000000              said  14315.000000  19.0000  19.0000
3207   Default   6295.000000            crypto   6295.000000  18.0000  18.0000
3219   Default   7432.000000  cryptocurrencies   7432.000000  17.0000  17.0000
7950   Default  11884.000000            market  11884.000000  16.0000  16.0000
7018   Default   5994.000000         investors   5994.000000  15.0000  15.0000
5236   Default  12861.000000         financial  12861.000000  14.0000  14.0000
12593  Default   5608.000000           support   5608.000000  13.0000  13.0000
3294   Default  17698.000000          currency  17698.000000  12.0000  12.0000
8229   Default   3566.000000            miners   3566.000000  11.0000  11.0000
9581   Default   6799.000000          platform   6799.000000  10.0000  10.0000
2401   Default   2518.000000          coinbase   2518.000000   9.0000   9.0000
2194   Default   3445.000000             china   3445.000000   8.0000   8.0000
3313   Default   4566.000000         customers   4566.000000   7.0000   7.0000
8618   Default   8339.000000           network   8339.000000   6.0000   6.0000
9378   Default  12898.000000            people  12898.000000   5.0000   5.0000
8211   Default   5671.000000           million   5671.000000   4.0000   4.0000
11452  Default   4852.000000          security   4852.000000   3.0000   3.0000
3380   Default   6727.000000              data   6727.000000   2.0000   2.0000
9334   Default   5573.000000          payments   5573.000000   1.0000   1.0000
...        ...           ...               ...           ...      ...      ...
14011  Topic20     50.282135          voorhees     70.595626   3.9225  -6.7796
5676   Topic20    107.947407             garza    234.989911   3.4839  -6.0156
13301  Topic20    134.536632            trezor    351.236510   3.3021  -5.7955
1054